# Intro to Generative Adversarial Networks (GANs)

In this workshop we will be learning what a GAN is and where they are used. We will be building and training a simple GAN to
colour black and white images.

For this workshop we will be using IBM Watson Studio, an interactive online Python notebook.

The machine learning library used for this is Torch.

In [ ]:
### If you are running on a CPU instance (e.g. a default lite/free plan of Watson Studio) 
### then pyTorch will not be installed by default and we need to install it, note this may
### take a few minutes to complete
try:
    import torch
except ImportError:
    !conda install pytorch-cpu -c pytorch

In [ ]:
import argparse
import os, math
import io
from tqdm import tqdm
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch
import requests
import warnings
import zipfile
import numpy as np

from matplotlib import pyplot as plt

from PIL import Image
from skimage.color import rgb2lab, lab2rgb, rgb2yuv, yuv2rgb

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("Device", device)

warnings.filterwarnings("ignore")

## Load in the Data

We need to be able to load in data for training. There is a zip file with a selection of images taken from the 'CelebA' 
dataset.

In [ ]:
FACES_URL = "https://github.com/IBMDeveloperUK/Colourise-GAN-Workshop/raw/master/data/faces64_1000.zip"
# If you have a GPU and want more data to train on, try the larger dataset below
#FACES_URL = "https://github.com/IBMDeveloperUK/Colourise-GAN-Workshop/raw/master/data/faces64_10000.zip"

# These values affect the speed and rate of learning
batch_size = 32
lr = 1e-3
b1 = 0.5
b2 = 0.999
pixel_loss_weights = 100.0

# Number of epochs to run. On a CPU instance it will take a few minutes per epoch
# if you are running on a GPU try increasing this to 10-30
n_epochs = 1

# A number of torch dataset iterators that can download images and get them
# in the right format for what we need.
class rgb_img_data(Dataset):
    def __init__(self, url):
        r = requests.get(url)
        self.z = zipfile.ZipFile(io.BytesIO(r.content))
        self.files = self.z.namelist()

    def __len__(self):
        return len(self.files)

    def __getitem__(self, index):
        filename = self.files[index]
        img = Image.open(self.z.open(filename))
        if img.mode != 'RGB':
          img = img.convert('RGB')
        return img

class lab_img_data(Dataset):
    def __init__(self, rgb_img_gen):
        self.rgb_img_gen = rgb_img_gen

    def __len__(self):
        return len(self.rgb_img_gen)

    def __getitem__(self, index):
        img = self.rgb_img_gen[index]
        lab = rgb2lab(img)
        return lab
    
class tensor_img_data(Dataset):
    def __init__(self, lab_img_gen):
        self.lab_img_gen = lab_img_gen

    def __len__(self):
        return len(self.lab_img_gen)

    def __getitem__(self, index):
        lab = self.lab_img_gen[index]
        l = lab[...,0] / 50 - 1
        a_t = lab[...,1] / 128
        b_t = lab[...,2] / 128
        return torch.Tensor(np.expand_dims(l,axis=0)),torch.Tensor(np.stack([a_t,b_t],axis=0))

In [ ]:
n_samples = 5

# Get the iterator for the source RGB images
rgb_images = rgb_img_data(FACES_URL)
fig = plt.figure(figsize=[12,4])
fig.suptitle('Original RGB images') 
for i in range(5):
    plt.subplot(2, n_samples, 1 + i)
    plt.axis('off')
    
    rgb_img = np.asarray(rgb_images[i])

    plt.imshow(rgb_img)

plt.show()

# create an iterator for LAB-format images so we can display the greyscale versions
lab_images = lab_img_data(rgb_images)
fig = plt.figure(figsize=[12,4])
fig.suptitle('Greyscale images') 
for i in range(n_samples):
    plt.subplot(2, n_samples, 1 + i)
    plt.axis('off')

    yuv_img = np.asarray(lab_images[i])

    plt.imshow(yuv_img[:,:,0], cmap='gray')
    
plt.show()

## Define the Models

We need to define a model for both the "generator" and the "discriminator". We define a GeneratorEncoderBlock and GeneratorDecoderBlock as we re-use the blocks multiple times in the networks.

The generator network is a U-Net, which means that it has a "contraction" followed by an "expansion" phase (like an autoencoder) but that it has a number of skip connections to join the corresponding blocks on each side of the U. 

In [ ]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)

# Calculate asymmetric TensorFlow-like 'SAME' padding for a convolution
def get_same_padding(x, kernel_sz, stride, dialation):
    return max((math.ceil(x / stride) - 1) * stride + (kernel_sz - 1) * dialation + 1 - x, 0)

# Dynamically pad input x with 'SAME' padding for conv with specified args
def pad_same(x, kernel_sz, stride, dialation=(1, 1), value=0.):
    ih, iw = x.size()[-2:]
    pad_h, pad_w = get_same_padding(ih, kernel_sz[0], stride[0], dialation[0]), get_same_padding(iw, kernel_sz[1],
                                                                                                 stride[1],
                                                                                                 dialation[1])
    if pad_h > 0 or pad_w > 0:
        x = F.pad(x, [pad_w // 2, pad_w - pad_w // 2, pad_h // 2, pad_h - pad_h // 2], value=value)
    return x

class GeneratorEncoderBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride, batch_norm=True, kernel_size=4):
        super(GeneratorEncoderBlock, self).__init__()

        layer_modules = [
            nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride)
        ]
        if batch_norm == True:
            layer_modules.append(nn.BatchNorm2d(out_channels))
        layer_modules.append(nn.LeakyReLU(negative_slope=0.2))
        self.encoder = nn.Sequential(*layer_modules)
        self.stride = stride
        self.kernel_size = kernel_size

    def forward(self, x):
        x = pad_same(x, kernel_sz=(self.kernel_size, self.kernel_size), stride=(self.stride, self.stride))
        return self.encoder(x)

class GeneratorDecoderBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride, batch_norm=True, kernel_size=4, padding= 1):
        super(GeneratorDecoderBlock, self).__init__()
        layer_modules = [
            nn.ConvTranspose2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding)
        ]
        if batch_norm == True:
            layer_modules.append(nn.BatchNorm2d(out_channels))
        layer_modules.append(nn.ReLU())
        self.decoder = nn.Sequential(*layer_modules)
        self.stride = stride
        self.kernel_size = kernel_size
        self.padding = padding

    def forward(self, x):
        x = self.decoder(x)
        return x

class Generator(nn.Module):
    def __init__(self, name, output_channels=2, training=True):
        super(Generator, self).__init__()
        self.name = name
        self.output_channels = output_channels
        self.training = training
        self.encoder_block_1 = GeneratorEncoderBlock(1, 64, 1)
        self.encoder_block_2 = GeneratorEncoderBlock(64, 64, 2)
        self.encoder_block_3 = GeneratorEncoderBlock(64, 128, 2)
        self.encoder_block_4 = GeneratorEncoderBlock(128, 256, 2)
        self.encoder_block_5 = GeneratorEncoderBlock(256, 512, 2)
        self.encoder_block_6 = GeneratorEncoderBlock(512, 512, 2)

        self.decoder_block_1 = GeneratorDecoderBlock(512, 512, 2)
        self.decoder_block_2 = GeneratorDecoderBlock(512, 256, 2)
        self.decoder_block_3 = GeneratorDecoderBlock(256, 128, 2)
        self.decoder_block_4 = GeneratorDecoderBlock(128, 64, 2)
        self.decoder_block_5 = GeneratorDecoderBlock(64, 64, 2)
        self.last_layer = nn.Conv2d(64, self.output_channels, kernel_size=1, stride=1)

    def forward(self, inputs):
        layers = []
        output = self.encoder_block_1(inputs)
        layers.append(output)
        output = self.encoder_block_2(output)
        layers.append(output)
        output = self.encoder_block_3(output)
        layers.append(output)
        output = self.encoder_block_4(output)
        layers.append(output)
        output = self.encoder_block_5(output)
        layers.append(output)
        output = self.encoder_block_6(output)
        layers.append(output)

        output = self.decoder_block_1(output)
        output = output + layers[len(layers) - 0 - 2]

        output = self.decoder_block_2(output)
        output = output + layers[len(layers) - 1 - 2]

        output = self.decoder_block_3(output)
        output = output + layers[len(layers) - 2 - 2]

        output = self.decoder_block_4(output)
        output = output + layers[len(layers) - 3 - 2]

        output = self.decoder_block_5(output)
        output = output + layers[len(layers) - 4 - 2]

        output = torch.tanh(self.last_layer(output))
        return output

class Discriminator(nn.Module):
    def __init__(self, name, training=True):
        super(Discriminator, self).__init__()
        self.name = name
        self.training = training
        self.discriminator_block_1 = GeneratorEncoderBlock(3, 64, 2, False)
        self.discriminator_block_2 = GeneratorEncoderBlock(64, 128, 2)
        self.discriminator_block_3 = GeneratorEncoderBlock(128, 256, 2)
        self.discriminator_block_4 = GeneratorEncoderBlock(256, 512, 2)
        self.last_layer = nn.Conv2d(512, 1, kernel_size=4, stride=1)

    def forward(self, inputs):
        output = self.discriminator_block_1(inputs)
        output = self.discriminator_block_2(output)
        output = self.discriminator_block_3(output)
        output = self.discriminator_block_4(output)
        output = torch.sigmoid(self.last_layer(output))
        return output

# Training

This is where the actual training occurs. We create instances of our networks and the data loader and then loop through a number of epochs and then train both the generator and the disciminator. On a CPU-based instance this will take about 8-10 minutes per epoch.

In [ ]:
# Initialize generator and discriminator
generator = Generator('G', training=True).to(device)
discriminator = Discriminator('D', training=True).to(device)

# Loss function
adversarial_loss = torch.nn.BCELoss()

# Initialize weights
generator.apply(weights_init_normal)
discriminator.apply(weights_init_normal)

# Instantiate the data loader
rgb_images = rgb_img_data(FACES_URL)
lab_images = lab_img_data(rgb_images)
trainset = tensor_img_data(lab_images)
params = {'batch_size': 32,
          'shuffle': True,
          'drop_last': True,
          'num_workers': 16}
trainloader = DataLoader(trainset, **params)

# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(b1, b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr/10, betas=(b1, b2))


# ----------
#  Training
# ----------

for epoch in tqdm(range(n_epochs), total=n_epochs):
    g_losses = []
    d_losses = []
    for i, (l, ab) in tqdm(enumerate(trainloader), total=int(len(trainset) / batch_size)):
        # Adversarial ground truths
        valid = Variable(torch.Tensor(l.size(0), 1).fill_(1.0), requires_grad=False).to(device)
        fake = Variable(torch.Tensor(l.size(0), 1).fill_(0.0), requires_grad=False).to(device)

        lvar = Variable(l).to(device)
        abvar = Variable(ab).to(device)
        real_imgs = torch.cat([lvar,abvar],dim=1)

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()
        
        # Generate a batch of images
        abgen = generator(lvar)
        gen_imgs = torch.cat([lvar.detach(),abgen],dim=1)

        # Loss measures generator's ability to fool the discriminator
        g_loss_gan = adversarial_loss(discriminator(gen_imgs.detach()), valid)
        g_loss = g_loss_gan + pixel_loss_weights * torch.mean((abvar-abgen)**2)

        g_losses.append(g_loss)
        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()
        
        # Measure discriminator's ability to classify real from generated samples
        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2

        d_losses.append(d_loss)
        d_loss.backward()
        optimizer_D.step()
        if i and (i + 1) % 50 == 0:
            print("Epoch: %d, Iter: %d,\tG loss: %.2f,\tD loss: %.2f" % (epoch, i + 1, g_loss.item(), d_loss.item()))

In [ ]:
# URL of a test image
test_img_url = "https://github.com/IBMDeveloperUK/Colourise-GAN-Workshop/raw/master/data/matt_rgb.jpg"

# Fetch the test image, resize it to correct size and convert it to numpy array
r = requests.get(test_img_url)
test_img = np.asarray(Image.open(io.BytesIO(r.content)).resize((64,64)))

# Original RGB image
plt.figure(figsize=[12,6])
plt.subplot(1, 3, 1)
plt.axis('off')
plt.title('Original RGB')
plt.imshow(test_img)

# Greyscale image
plt.subplot(1, 3, 2)
plt.axis('off')
plt.title('Original Greyscale')
test_lab = rgb2lab(test_img)
plt.imshow(test_lab[:,:,0], cmap='gray')

# Generated image
test_inf = test_lab[...,0].reshape(1,1,64,64)
# We need to scale the L value of the LAB image to -1..1 for the neural network
test_var = Variable(torch.Tensor(test_inf / 50 - 1)).to(device)

# Generate the image by passing in the greyscale version
test_res = generator(test_var)
ab=test_res.cpu().detach().numpy()

# We need to scale the AB values returned from the generator to be in the range -128..128
ab[:,0,:,:] *= 128
ab[:,1,:,:] *= 128

# Convert from LAB back to RGB format.
gen_lab = np.concatenate([test_inf,ab],axis=1).reshape(3,64,64)
gen_rgb = lab2rgb(gen_lab.transpose(1,2,0))
gen_rgb = np.clip(gen_rgb, 0, 1)

plt.subplot(1, 3, 3)
plt.axis('off')
plt.title('Generated RGB')
plt.imshow(gen_rgb)

plt.show()